# Topic Model of the State Archives of Assyria Letters

This is a work-in-progress Notebook. The number of topics is set with the ntopics variable. There are almost 2500 letters, most of them brief or very brief. The topic model produced from the entire corpus (with 15 or 25 topics) is hard to interpret. One reason may be that the texts are too short. A brief or broken letter with just a few words that happen to be important in a certain topic may come out as one of the most important documents in that topic (because a high percentage of that letter resides in the topic). One solution may be to group (aggregate) letters by metadata, in the file SAAO.csv. One could group the letters by Chapter number (this referes to chapters in the original book publications - assuming that those chapters group texts that are related in a meaningful way)


In [1]:
import pandas as pd
import glob
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Read in Data
First read the directory with the State Archives of Assyria letters. These files contain lemmatization in ORACC. The texts list lemmatizations per line.

In [2]:
path =r'../Scrape-Oracc/Output/' # use your path
allFiles = glob.glob(path + "/saao" + "*.txt")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
saao_data = pd.concat(list_)
saao_data.head()

,id_text,text_name,l_no,text
0,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,o 1,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
1,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,o 2,šulmu[completeness]N ana[to]PRP libbu[interior...
2,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,o 3,ša[that]REL šapāru[send]V mā[saying]PRP māru[s...
3,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,o 4,Muskaya[Phrygian]EN ina[in]PRP muhhu[skull]N a...
4,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,o 5,Quwaya[from-Quwe]EN ša[that]REL Urik[1]PN ana[...


# Collapse lines to one row per document
In order to transform this DataFrame into a proper input for topic modeling we need to discard the column `l_no` and concatenate all the text that belongs to one letter in a single row. Some lines have no content in the `text` column - these lines need to be dropped.

First select the relevant columns and drop the rows that have no text content.

In [3]:
saao_data = saao_data[['id_text', 'text_name', 'text']]
saao_data = saao_data.dropna()
saao_data.head()

,id_text,text_name,text
0,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
1,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,šulmu[completeness]N ana[to]PRP libbu[interior...
2,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,ša[that]REL šapāru[send]V mā[saying]PRP māru[s...
3,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,Muskaya[Phrygian]EN ina[in]PRP muhhu[skull]N a...
4,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,Quwaya[from-Quwe]EN ša[that]REL Urik[1]PN ana[...


Group the rows by `id_text` and apply the `join` function to the `text` column. Transform the aggregated data into a new DataFrame.

In [4]:
saao_byletter = saao_data['text'].groupby(saao_data['id_text']).apply(' '.join)
saao_byletter_df = pd.DataFrame(saao_byletter)
saao_byletter_df.head()

,text
id_text,
saao/saa01/P224485,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
saao/saa01/P313416,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313417,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313425,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313427,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...


Create a DataFrame of `id_text` and `text_name` equivalencies, with `id_text` set as index (row names). Then merge this DataFrame with the the `saao_bytext_df` using the indexes.

In [5]:
saao_id_names = saao_data[['id_text', 'text_name']].drop_duplicates().set_index('id_text')
saao_data_df = pd.merge(saao_id_names, saao_byletter_df, right_index=True, left_index=True)
saao_data_df.head()

,text_name,text
id_text,,
saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
saao/saa01/P313416,SAA 01 158. Gold and Silver Objects Sent to th...,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313417,SAA 01 233. More Land to Bel-duri (CT 53 002),ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313425,SAA 01 179. No Iron to the Arabs! (CT 53 010),ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313427,SAA 01 152. The Affair of Gidgidanu and His Br...,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...


In [6]:
from gensim import corpora, models, utils
import gensim

# Test corpus: first 100 texts
[For test purposes only the first 100 documents are admitted.]

In [7]:
#saao_data_df = saao_data_df[:100]
documents = saao_data_df['text']

# Tokenize and POS-filter
Tokenization is done by splitting on white space. The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [8]:
posfilter = [']n', ']v', 'en']
texts = [[word for word in document.lower().split() if word[-2:] in posfilter] for document in documents]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those have already been filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.

In [9]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v']
texts = [[word for word in text if word not in stoplist] for text in texts]

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

In [12]:
seed = 42
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
ntopics = 25
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top words and their probabilities in all topics. Note: the topic numbers here are not the ones used below in the visualizations! (The topics are the same, but not their numbers).

In [13]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('dullu[trouble]n', 0.1121812841296506),
   ('karābu[pray]v', 0.040499207547792335),
   ('paqādu[entrust]v', 0.036143712977920815),
   ('īnu[eye]n', 0.033661343800300586),
   ('kittu[truth]n', 0.032879314751893284),
   ('piqittu[allocation]n', 0.026699805984847343),
   ('rabû[big-one]n', 0.021971284159811977),
   ('laššu[(there)-is-not]v', 0.020779152882955346),
   ('rēʾû[shepherd]n', 0.020458346278180297),
   ('hīṭu[error]n', 0.020034071944625781)]),
 (1,
  [('wadû[know]v', 0.048049745811283011),
   ('ēkallu[palace]n', 0.037254265422598916),
   ('rēšu[head]n', 0.027519841737613647),
   ('qātu[hand]n', 0.027282992058910462),
   ('šadādu[drag]v', 0.024134800889258321),
   ('galû[be(come)-deported]v', 0.023200817539834905),
   ('šēpu[foot]n', 0.022194213538224549),
   ('ahu[brother]n', 0.021911438198444683),
   ('haṭû[do-wrong]v', 0.020366900221906712),
   ('harāpu[be(come)-early]v', 0.020272897913267234)]),
 (2,
  [('sinništu[woman]n', 0.083405644096967324),
   ('gušūru[tree-tru

# pyLDAvis
Use pyLDAvis to visualize the topic model. By default, pyLDAvis will order the topics by [prevalence](https://github.com/bmabey/pyLDAvis/issues/59) (topic 1 is the most prevalent topic). That means that the topic numbers in the visualization do not agree with the topic numbers in the lda model. To prevent this behaviour one may use `sort_topics=False` in the `prepare` command. The advantage of ordering the topics by prevalence, however, is that new instances of the lda model are more comparable (that is, the same topic will receive the same number). Note that the library was written in Java for R, and so the numbering in the visualization begins with 1 (not with 0). The topic numbers in the Document/Topic and Topic/Term matrices below will be adjusted to be compatible with the pyLDAvis visualization.

In [14]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, 'saao.html')
pyLDAvis.display(vis)

//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
//anaconda/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationW

# Create Document/Topic Table
Create a DataFrame that gives the probability of each topic for each document.

In [15]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
df_list = [pd.DataFrame(list_of_probabilities[i]).transpose() for i in range(len(corpus))]
doc_topic_df = pd.concat(df_list)
doc_topic_df = doc_topic_df.set_index(saao_data_df.index)
doc_topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
id_text,,,,,,,,,,,,,,,,,,,,,
saao/saa01/P224485,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.989677,0.000430,0.000430,...,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430
saao/saa01/P313416,0.061859,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,...,0.001176,0.001176,0.001176,0.881670,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176
saao/saa01/P313417,0.000870,0.000870,0.000870,0.000870,0.000870,0.000870,0.243439,0.159442,0.000870,0.036087,...,0.000870,0.000870,0.000870,0.000870,0.000870,0.187818,0.247436,0.000870,0.000870,0.000870
saao/saa01/P313425,0.035022,0.000541,0.000541,0.000541,0.000541,0.000541,0.067795,0.000541,0.000541,0.029239,...,0.000541,0.079524,0.000541,0.094559,0.065016,0.065758,0.385601,0.000541,0.000541,0.000541
saao/saa01/P313427,0.383370,0.055064,0.046946,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.144303,...,0.001667,0.001667,0.001667,0.001667,0.280091,0.001667,0.001667,0.001667,0.001667,0.001667


Reorder the topics (columns) according to prevalence and rename the topics `topic 1` to `topic n` in accordance with the pyLDAvis visualization.

In [16]:
topics = ['topic ' + str(i+1) for i in range(ntopics)]

In [17]:
doc_topic_df.columns = topics
doc_topic_df['text_name'] = saao_data_df['text_name']
doc_topic_df.head()

,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,...,topic 17,topic 18,topic 19,topic 20,topic 21,topic 22,topic 23,topic 24,topic 25,text_name
id_text,,,,,,,,,,,,,,,,,,,,,
saao/saa01/P224485,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.989677,0.000430,0.000430,...,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,0.000430,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...
saao/saa01/P313416,0.061859,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,...,0.001176,0.001176,0.881670,0.001176,0.001176,0.001176,0.001176,0.001176,0.001176,SAA 01 158. Gold and Silver Objects Sent to th...
saao/saa01/P313417,0.000870,0.000870,0.000870,0.000870,0.000870,0.000870,0.243439,0.159442,0.000870,0.036087,...,0.000870,0.000870,0.000870,0.000870,0.187818,0.247436,0.000870,0.000870,0.000870,SAA 01 233. More Land to Bel-duri (CT 53 002)
saao/saa01/P313425,0.035022,0.000541,0.000541,0.000541,0.000541,0.000541,0.067795,0.000541,0.000541,0.029239,...,0.079524,0.000541,0.094559,0.065016,0.065758,0.385601,0.000541,0.000541,0.000541,SAA 01 179. No Iron to the Arabs! (CT 53 010)
saao/saa01/P313427,0.383370,0.055064,0.046946,0.001667,0.001667,0.001667,0.001667,0.001667,0.001667,0.144303,...,0.001667,0.001667,0.001667,0.280091,0.001667,0.001667,0.001667,0.001667,0.001667,SAA 01 152. The Affair of Gidgidanu and His Br...


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [18]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. Each Dataframe is transposed and the DataFrames are concatenated to a single DataFrame. 

In [19]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0).transpose() for i in range(0, ntopics)]
topic_term_df = pd.concat(topic_term_list, ignore_index=True)
topic_term_df['topics'] = topics
topic_term_df = topic_term_df.set_index('topics')

The topics (rows) are numbered 0-14 in the (random) order of the lda model. The array `prevalences` is used to re-arrange the topics in the order of prevalence (as is done above for the document/topic matrix). The re-arranged topics (rows) are now re-labeled `topic 1` to `topic 15` in accordance with their numbering in the pyLDAvis visualization.

In [20]:
topic_term_df['ēkallu[palace]n']

topics
topic 1     0.000023
topic 2     0.037254
topic 3     0.043689
topic 4     0.008879
topic 5     0.000030
topic 6     0.007247
topic 7     0.000038
topic 8     0.002198
topic 9     0.000036
topic 10    0.000026
topic 11    0.013788
topic 12    0.014339
topic 13    0.000037
topic 14    0.003643
topic 15    0.002484
topic 16    0.000039
topic 17    0.027082
topic 18    0.014091
topic 19    0.020451
topic 20    0.012933
topic 21    0.000033
topic 22    0.019417
topic 23    0.039338
topic 24    0.007221
topic 25    0.000044
Name: ēkallu[palace]n, dtype: float64

The visualization below plots all the documents according to their distances (using Multi-Dimensional Scaling) in the Document/Topic DataFrame. Each document (dot) is colored according to the most prevalent topic and the size of the dot represents the probability of the most prevalent topic in that document.

In [21]:
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS

Compute the distances between each of the documents. Use either the Document/Topic Dataframe or the Document/Term Dataframe (constructed below) to measure distance.

In [22]:
cv = CountVectorizer(analyzer='word', token_pattern=r'[^ ]+', vocabulary=list(topic_term_df.columns.values))
saao_dtm = cv.fit_transform(saao_data_df['text'])
saao_dtm_df = pd.DataFrame(saao_dtm.toarray(), columns = cv.get_feature_names(), index = saao_data_df.index.values)
saao_dtm_df.head()

,abati[(meaning-unknown)]n,abbūtu[fatherhood]n,abdudaya[abdudaean]en,abiktu[defeat]n,abku[captive]n,abnu[stone]n,abu[father]n,abullu[(city)-gate]n,abāku[lead-away]v,abālu[dry-(up)]v,...,ṭābtu[goodness]n,ṭābtu[salt]n,ṭābtānu[doer-of-good]n,ṭābtūtu[goodwill]n,ṭātu[bribe]n,ṭēmu[(fore)thought]n,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭūbu[goodness]n
saao/saa01/P224485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
saao/saa01/P313416,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
saao/saa01/P313425,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313427,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#dist = squareform(pdist(saao_dtm_df))
dist = squareform(pdist(doc_topic_df.drop('text_name', axis=1)))

Compute the position of each document using Multi-Dimensional Scaling. The variable `pos` holds the `x` and `y`  coordinates.

In [24]:
mds = MDS(n_components=2, max_iter=3000,
      random_state=seed, dissimilarity="precomputed", n_jobs=1)

pos = mds.fit(dist).embedding_

Create a list of x values (coordinates) and a list of y values.

In [25]:
pos_x = [x for x, y in pos]
pos_y = [y for x, y in pos]

Create lists of the most prevalent topic, the probability of the most prevalent topic, and the text name for each document.

In [26]:
prevalent_topic = doc_topic_df.drop('text_name', axis=1).idxmax(axis=1)
probability = [doc_topic_df.ix[i][prevalent_topic[i]] for i in range(0, len(corpus))]
text_name = [name for name in doc_topic_df['text_name']]

In [27]:
len(prevalent_topic)

2415

Import bokeh and draw the visualization

In [28]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool
from bokeh.plotting import figure, output_file, output_notebook, show

output_notebook()

p = figure(
    plot_width=800, plot_height=800,
    tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
    title="Topic Distribution MDS\nSize of the circle represents prevalence of the topic")
p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))

colormap = {'topic 1': "orange", 'topic 2': "olive", 'topic 3': "firebrick", 
          'topic 4': "gold", 'topic 5': "red", 'topic 6': "black", 'topic 7': "green", 
          'topic 8': "blue", 'topic 9': "purple", 'topic 10': "grey", 'topic 11': "yellow", 
          'topic 12': "indigo", 'topic 13': "blueviolet", 'topic 14': "beige", 'topic 15':"navy", 'topic 16': 'orange',
           'topic 17': 'olive', 'topic 18': 'firebrick', 'topic 19': 'gold', 'topic 20': 'red', 'topic 21': 'black', 
            'topic 22': 'green', 'topic 23': 'blue', 'topic 24': 'purple', 'topic 25': 'grey'}
colors = [colormap[n] for n in prevalent_topic]

source = ColumnDataSource(data=dict(
        x=pos_x,
        y=pos_y,
        id_text=list(doc_topic_df.index.values),
        size = probability/max(probability)*25,
        probability = probability,
        topic = prevalent_topic,
        text_name = text_name
    ))

p.circle('x', 'y', color=colors, fill_alpha=0.5, size='size', source=source)

url = "http://oracc.museum.upenn.edu/@id_text"
    

taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)

Loading BokehJS ...

The following graph uses a slider to choose one topic. The idea is to color the dots (documents) that have high prevalence for that topic and leave the other dots gray (or: use a high fill_alpha for the chosen topic and a low one for the ones that are backgrounded). Since the dots link to the online editions of these documents, that would make it easier to explore a topic. The Java code (in CustomJS) in the `callback` function currently does not work.

In [50]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, CustomJS
from bokeh.models.widgets import Slider
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import widgetbox
output_notebook()

p = figure(
    plot_width=800, plot_height=800,
    tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
    title="Topic Distribution MDS\nSize of the circle represents prevalence of the topic")
p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))

colormap = {'topic 1': "orange", 'topic 2': "olive", 'topic 3': "firebrick", 
          'topic 4': "gold", 'topic 5': "red", 'topic 6': "black", 'topic 7': "green", 
          'topic 8': "blue", 'topic 9': "purple", 'topic 10': "grey", 'topic 11': "yellow", 
          'topic 12': "indigo", 'topic 13': "blueviolet", 'topic 14': "beige", 'topic 15':"navy", 'topic 16': 'orange',
           'topic 17': 'olive', 'topic 18': 'firebrick', 'topic 19': 'gold', 'topic 20': 'red', 'topic 21': 'black', 
            'topic 22': 'green', 'topic 23': 'blue', 'topic 24': 'purple', 'topic 25': 'grey'}
colors = [colormap[n] for n in prevalent_topic]

source = ColumnDataSource(data=dict(
        x=pos_x,
        y=pos_y,
        id_text=list(doc_topic_df.index.values),
        size = probability/max(probability)*25,
        probability = probability,
        topic = prevalent_topic,
        color = colors,
        text_name = text_name
    ))

p.circle('x', 'y', color=colors, fill_alpha=0.5, size='size', source=source)

callback = CustomJS(args=dict(source=source), code = """
    var data = source.data;
    var Topic = topic.value;
    x = data['x']
    y = data['y']
    color = data['color']
    for (i=0; i<ntopics; i++)
    {
    color = color[Topic]
    }
    source.trigger('change')

""")

topic_slider = Slider(start=1, end = ntopics, value=1, step=1, title = 'Topic', callback=callback)
callback.args['topic'] = topic_slider


url = "http://oracc.museum.upenn.edu/@id_text"

taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(widgetbox(topic_slider))
show(p)

Loading BokehJS ...